In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

import tiktoken
from llama_index import SimpleDirectoryReader, GPTVectorStoreIndex
from llama_index import OpenAIEmbedding, ServiceContext
from llama_index.node_parser import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter

load_dotenv()

True

In [2]:
docs_dir = Path('/home/jinzy/work/automation/ask_gpt/testdocs')
embed_model = OpenAIEmbedding()

In [10]:
import PyPDF2

#filename = '/home/jinzy/work/automation/ask_gpt/documents/process-mining-0529/process-discovery-algorithms/Leemans_thesis_Hierarchical Process Mining.pdf'

def find_unprocessable_pdfs(directory):
    unprocessable_pdfs = []

    # Walk through the directory tree
    for root, dirs, files in os.walk(directory):
        for file in files:
            # If the file is a PDF
            if file.endswith('.pdf') or file.endswith('.PDF'):
                pdf_path = os.path.join(root, file)
                
                # Attempt to open the file with PyPDF2
                try:
                    with open(pdf_path, 'rb') as f:
                        PyPDF2.PdfReader(f)
                except:
                    print(pdf_path)
                    # If an error is raised, add it to the list of unprocessable files
                    unprocessable_pdfs.append(pdf_path)

    return unprocessable_pdfs

In [11]:
find_unprocessable_pdfs(docs_dir)

[]

In [12]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
documents = SimpleDirectoryReader(docs_dir, recursive=True, required_exts=['.docx', '.pdf'], errors='ignore').load_data()
print(len(documents))

3


In [13]:
from langchain import OpenAI
from llama_index import GPTVectorStoreIndex, LLMPredictor, PromptHelper

# set maximum input size
max_input_size = 4096
# set number of output tokens
num_outputs = 256
# set maximum chunk overlap
max_chunk_overlap = 30
# set chunk size limit
chunk_size_limit = 1000

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name="text-davinci-003", max_tokens=num_outputs))
prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

index = GPTVectorStoreIndex.from_documents(
        documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper
    )

# node_parser = SimpleNodeParser(text_splitter=text_splitter)
# service_context = ServiceContext.from_defaults(
#     embed_model=embed_model, node_parser=node_parser
# )
# index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

In [14]:
query_engine = index.as_query_engine()
query_engine.query('请解释一下什么是任务挖掘?')

Response(response='\n任务挖掘（Process Mining）是一种利用事件日志（event log）来发现、分析和改进业务流程的技术。它可以帮助企业从企业各系统中捕捉信息，并创建事件日志和建立流程模型，以显示企业流程运行的真实情况，包括谁做的、花了多长时间，以及跟平均水平的偏离程度。它还可', source_nodes=[NodeWithScore(node=Node(text='的首席科学顾问。•流程挖掘技术可以帮助企业轻松地从企业各系统中捕捉信息，并提供关于主要流程执行情况的详细且以数据为驱动的信息。当完成数据对接后，流程挖掘技术会通过创建事件日志和建立流程模型的方式，显示企业流程运行的真实情况，包括谁做的、花了多长时间，以及跟平均水平的偏离程度。流程分析所创建的关键绩效指标（KPIs），使公司能够将重点放在优先级步骤或活动上进行改进。人工智能算法可以检测到流程绩效出现变化的根本原因——例如，流程挖掘可以发现，每次新客户需要进行信用检查时，都会导致订单到现金流程（OTC流程）效率的减慢。•通过流程挖掘，企业去了解当前流程运行状态，发现流程存在问题的过程不再是昂贵低效的手工投入，而是快捷高效的自动处理。甚至，针对管理层和普通用户等不同使用者，流程挖掘提供了可灵活切换分析颗粒度的可能，让企业所有人，都能够围绕自己所关心的问题，了解情况，解决问题。\n\n第⼆章  流程挖掘的技术发展\n信息化的发展使得企业在执行实际业务过程中产生了大量的事件日志（event', doc_id='54a4d182-edff-4450-b15c-83c9094cfbdf', embedding=None, doc_hash='cd5539e3ef806f3fcc6a663669445c4461f5854943fad63cd3d7f5173f18d5ba', extra_info=None, node_info={'start': 4159, 'end': 4602}, relationships={<DocumentRelationship.SOURCE: '1'>: '4650c543-e75a-43bb-a0ba-9dc072d5c3ff', <DocumentRelationship.PREVIOUS: '2'>: '8f76aafc-256a-4229-9b

In [15]:
query_engine = index.as_query_engine()
query_engine.query('了解一下RPM项目建设')

Response(response='\niS-RPM是一个RPA解决方案，它可以帮助企业实现RPA自动化，从而提高流程自动化的收益，加快RPA建设的进程，实现RPA行业的规模化变革。iS-RPM采用自上而下和自下而上结合的方式，持续发现自动化机会，降低自动化门槛，缩短部署时间，提升自动化收益。它', source_nodes=[NodeWithScore(node=Node(text=' 另一方面，可以更加全面的了解业务场景，不局限于人的主观经验。      RPA工程文件  除了导出RPA需求文档，iS-RPM还支持一键导出RPA工程文档。  如果企业在使用iS-RPM之前，已经使用了艺赛旗的iS-RPA设计器。那么可以将导出“.irpm”文件导入到设计器中，并且一键转换成RPA工程，把组件运行需要的参数自动生成，得到我们右边这张图的流程骨架。我们提供这样的一个骨架流程能帮助RPA开发者快速生成流程的重要步骤节点，加快实施交付的进度。             场景&案例  4.1某能源企业案例  该企业内部业务系统繁多复杂。作为国内共享服务专业化公司，该企业的共享服务中心全面承接集团企业的财务、人事、IT、党建和商旅业务，每日需要集中处理大量的事务性工作，给业务处理的及时性和准确性带来了不小挑战。因此，亟需通过自动化的方式减轻人员工作压力。由此，共享中心开启RPA自动化之旅，并结合任务挖掘新技术在智能自动化方面率先探索并取得成效。      4.2某运营商案例 ', doc_id='2bfbcbd0-6f4c-43c3-9569-ac5e958e4e2e', embedding=None, doc_hash='92b87aad80768da9aa5a763b20f7716a1e2053c5b9f7b950db3a63a985360bf2', extra_info=None, node_info={'start': 7464, 'end': 7911}, relationships={<DocumentRelationship.SOURCE: '1'>: 'f9a4d5a2-f090-4114-a7ad-166b6d3a6ccb', <DocumentRelationship.PREVIOUS: '2'>: '56a359b4-48e4-4d03-b1c0-52e

In [16]:
len(_.response)

129

In [7]:
query_engine = index.as_query_engine()
r = query_engine.query('请解释一下什么是任务挖掘?')
r.response

'\n任务挖掘是一种技术，旨在通过收集和分析用户交互行为来了解他们如何操作任务。它可以帮助企业发现自动化机会，减少流程差距，提供流程违规警报，加快自动化交付周期。任务挖掘与流程挖掘互相补充，以提供基于事实的见解并支持流程改进。'

In [8]:
r.source_nodes

[NodeWithScore(node=Node(text='流程挖掘原理\n数据采集\n可视化\n 映射、转换\nEvent end\nCase IDActivity\nA\nB\nC\n...\nABCAB\nC2021/11/2 10:00\n2021/10/17 10:00 2021/11/2 10:41\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\n能够通过收集和分析用户交互行为来了解他们如何操作任务。这样的深入了解可以帮助\n企业实现自动化和改进流程。\n任务挖掘\n任务记录 任务分析 任务推荐\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\n任务挖掘价值\n发现自动化机会 减少流程差距\n流程违规警报 加快自动化交付周期\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\n流程挖掘、任务挖掘之间的关系\nTask Mining互相补充Process Mining\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\nÑÏ½û¸´ÖÆ\n案例分享： RPM项目建设概况\nRPM项目背景及历程\n中移在线上海分公司主要负责客户的业务咨询', doc_id='248cc9e3-72e7-427f-acfc-22c12046650d', embedding=None, doc_hash='d57ef0694d9d69319511d9142063a3ad15abf8a936c1268725a90d30a63147fb', extra_info=None, node_info={'start': 1124, 'end': 1767}, relationships={<DocumentRelationship.SOURCE: '1'>: '6af0fff5-aa85-

In [9]:
r.extra_info

{'248cc9e3-72e7-427f-acfc-22c12046650d': None,
 'afe3341a-54ab-422a-bfe5-d0405cdcc05b': None}

In [20]:
persist_directory = 'rpmv3'
index.storage_context.persist(persist_dir=persist_directory)

In [11]:
r = query_engine.query('请解释一下什么是流程挖掘?')
r.response

'\n流程挖掘是一种新兴学科，提供了一整套工具，以提供基于事实的见解并支持流程改进。它建立在流程模型驱动方法和数据挖掘（Data Mining）的基础上，旨在将事件数据转化为见解（Insight）和行动（Action）。流程挖掘是一种基于多视角模型的流程挖掘方法和'

In [12]:
len(r.response)

130

In [13]:
r.source_nodes

[NodeWithScore(node=Node(text=': \n流程挖掘是一门新兴学科，提供了一整  \n套工具，以提供基于事实的见解并支持\n流程改进。这门新学科建立在流程模型\n驱动方法和数据挖掘（ Data  Mining）的\n基础上。然而，流程挖掘不仅仅是现有\n方法的合并。 例如，现有的数据挖掘技术\n过于以数据为中心，无法全面了解组织中\n的端到端流程。 BI 工具侧重于简单的仪\n表盘和报告，而不是清晰的业务流程洞察\n力。 BPM  套件严重依赖于专家对理想化  \n流程进行建模，不能帮助大众理解现有流\n程。 图 1：流程挖掘之父范德阿尔斯特，威尔  \n \n他进一步指出流程挖掘作为数据和流程科学之间的超级粘合剂【2】（见图 2），其目标是将事\n件数据转化为见解 （Insight）和行动（Action）。流程挖掘是数据科学不可分割的一部分，数\n据的可用性和改进流程的愿望推动了流程挖掘。  \n \n图 2: 流程挖掘作为数据和流程科学之间的超级粘合剂（ 来源：【 2】） \n \n流程挖掘是一种在近几年中越来越受欢迎的技术。如图  3 所示，流程挖掘弥补了传统的、\n基于模型的流程分析', doc_id='afe3341a-54ab-422a-bfe5-d0405cdcc05b', embedding=None, doc_hash='78eca4237128d76a76d050cc7a79df32fc76983b3f5891d1340c9fcfe706a2ac', extra_info=None, node_info={'start': 982, 'end': 1446}, relationships={<DocumentRelationship.SOURCE: '1'>: '8a9622dc-6422-4b8d-bb80-4ec2c10a81d8', <DocumentRelationship.PREVIOUS: '2'>: 'c71d3610-d753-4c87-a53c-899b732dfcca', <DocumentRelationship.NEXT: '3'>: '6c5588d4-42bc-4879-80f0-10f690e10216'}), score=0.9090726781703465),
 NodeWithScor